In [150]:
from __future__ import annotations
import argparse
from types import SimpleNamespace
from typing import List, IO, Optional, Any, Tuple, Union
import logging
import time
from pathy import Pathy
from pathlib import Path
import sys

%load_ext autoreload
%autoreload 2


import ray
from smart_open import open as sopen
import os
import json
import tqdm

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [194]:
def recursive_collect_sents(d: dict):
    
    all_sents, all_ids = [], []
    
    if "sentences" in d:
        sents = d["sentences"]
        for s in sents:
            sent_id, txt = s["sentenceId"], " ".join(s["words"]).replace("\n","")
            all_ids.append(sent_id)
            all_sents.append(txt)
    
    else:
            for inner_d in d["body"]:
                inner_sents, inner_ids = recursive_collect_sents(inner_d)
                all_sents.extend(inner_sents)
                all_ids.extend(inner_ids)
    
    return all_sents, all_ids

def process_file(in_file_name, out_file_name):
    with sopen(in_file_name) as infh:
        with sopen(out_file_name, "w") as outfh:
            for line in tqdm.tqdm_notebook(infh):
                d = json.loads(line)
                sents,ids = recursive_collect_sents(d)
                for txt,sent_id in zip(sents, ids):
                    outfh.write(json.dumps({"text": txt, "sent_id": sent_id}) + "\n")
    return 1


@ray.remote
def process_file_ray(in_file_name, out_file_name):
    print("start process")
    return process_file(in_file_name, out_file_name)
    print("end process")

In [168]:
ray.shutdown()
ray.init(local_mode=True)
in_file = "gs://ai2i-us/SPIKE/datasets/annh/wikipedia/annh__AA_wiki_00.0002.jsonl.gz"
out_file = "gs://ai2i-us/SPIKE/datasets/text/wikipedia/text__AA_wiki_00.0002.jsonl.gz"
in_files = [in_file]
out_files = [out_file]

In [160]:
import os
import time
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = "./ai2-israel-d3744270e886.json"
#process_file(in_file, out_file)

:task_name:process_file_ray


0it [00:00, ?it/s]

0.0006060600280761719


In [30]:
d = json.loads(line)

In [193]:
ray.shutdown()
ray.init(local_mode=True)

# files
|G
in_file = "gs://ai2i-us/SPIKE/datasets/annh/wikipedia/annh__AA_wiki_00.0025.jsonl.gz"
out_file = "gs://ai2i-us/SPIKE/datasets/text/wikipedia/text__AA_wiki_00.0025.jsonl.gz"
in_files = [in_file]
out_files = [out_file]

# collect tasks

tasks = []
start = time.time()
for in_f,out_f in zip(in_files, out_files):
    tasks.append(process_file_ray.remote(in_f, out_f))
    
print("collection time", time.time() - start)    

# run tasks

start = time.time()
results = ray.get(tasks)
print("running time", time.time() - start)

:task_name:process_file_ray
start process


0it [00:00, ?it/s]

collection time 12.853028059005737
running time 0.0024902820587158203


In [162]:
start = time.time()
results = ray.get(tasks)
print(time.time() - start)

0.0006542205810546875


In [180]:
start = time.time()
process_file("annh__AA_wiki_00.2300.jsonl.gz", out_file)
print(time.time() - start)

FileNotFoundError: [Errno 2] No such file or directory: 'annh__AA_wiki_00.2300.jsonl.gz'